In [74]:
import torch
from torch import nn

In [75]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [76]:
class GoogleNet(nn.Module):
    def __init__(self, aux_logits=True, num_classes=1000):
        super().__init__()
        assert aux_logits == True or aux_logits == False
        self.aux_logits = aux_logits
        self.conv1 = conv_layer(3,
                                64,
                                kernel_size=7,
                                stride=2,
                                padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) 
        self.conv2 = conv_layer(
            64, 192, kernel_size=3, stride=1, padding=1
            )
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        #in_channels, out_1x1, reduced_3x3, out_3x3, reduced_5x5, out_5x5, out_1x1_pool
        self.inception3a = Inception_Block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception_Block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, 2, 1)
        
        self.inception4a = Inception_Block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception_Block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception_Block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception_Block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception_Block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, 2, 1)
        
        self.inception5a = Inception_Block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception_Block(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)
        
        if self.aux_logits:
            self.aux1 = Inception_Aux(512, num_classes)
            self.aux2 = Inception_Aux(528, num_classes)
        else:
            self.aux1 = self.aux2 = None
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        
        if self.aux_logits == True and self.training:
            aux1 = self.aux1(x)
            
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        
        if self.aux_logits == True and self.training:
            aux2 = self.aux2(x)
        
        x = self.inception4e(x)        
        x = self.maxpool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        
        x = x.reshape(x.shape[0], -1)
        
        x = self.dropout(x)
        x = self.fc(x)
        
        if self.aux_logits and self.training:
            return aux1, aux2, x
        else:
            return x 

In [77]:
class Inception_Block(nn.Module):
    def __init__(self, in_channels, out_1x1, reduced_3x3, out_3x3, reduced_5x5, out_5x5, out_1x1_pool):
        super().__init__()
        self.branch1 = conv_layer(in_channels, out_1x1, kernel_size=1)
        self.branch2 = nn.Sequential(
            conv_layer(in_channels, reduced_3x3, kernel_size=1),
            conv_layer(reduced_3x3, out_3x3, kernel_size=3, padding=1)                                     
                                    )
        self.branch3 = nn.Sequential(
            conv_layer(in_channels, reduced_5x5, kernel_size=1),
            conv_layer(reduced_5x5, out_5x5, kernel_size=5, padding=2)
            )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_layer(in_channels, out_1x1_pool, kernel_size=1)    
        )
        
    def forward(self, x):
        #B C W H
        x = torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)
        return x

In [78]:
class Inception_Aux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.7)
        self.avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = conv_layer(in_channels, 128, kernel_size=1)
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)        
        return x
        
        

In [79]:
class conv_layer(nn.Module):
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_features,out_features, **kwargs)
        self.batch_norm = nn.BatchNorm2d(out_features)
        
    def forward(self, x):
        return self.relu(self.batch_norm(self.conv(x))) 

In [85]:
if __name__ == "__main__":
    x = torch.randn(11,3,224,224).to(device)
    model = GoogleNet().to(device)
    print(model(x)[0].shape)

torch.Size([11, 1000])
